# **Summary**:

The purpose of the notebook is to summarize the solution for Team "Peng Wang + Datong Guo". The solution we used here is to stack 6 models. These 6 models are mainly NN models,the reason why they are seperated is due to different ways of dealing with dataset. The only Non-NN model used is TabNet. In order to comply with the GPU limit, the solution here is 5538.6 seconds, which includes all the trainning process. In addition, the 7200 seconds is for submission time, which means, if the notebook is commited successfully, there is a chance for notebook timeout error. Around 10% of the notebook should be reserved.

But if pretrainned models is used, the solution should have more potential. The below solution public score is 0.01823 and the private score is 0.01609.

# Exploratory Data Analysis

As there are several published notebooks for EDA analysis, we will not discussed all the analysis. Instead, we will discuss two parts, which seems not widely discussed.

**1.** Quantile Transformer for Genes and Cells, or Quantile Transformer only for Genes?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer

train_features = pd.read_csv('../input/lish-moa/train_features.csv')

In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

Genes_Mean = train_features[GENES + CELLS].mean()

fig, ax = plt.subplots(figsize=(15, 5))
ax.set_xlabel ("Normal", fontsize=18)
ax.set_ylabel ("Mean of Genes + Cells", fontsize=18);

Genes_Mean.plot(lw=3);

In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

transformed_all = train_features

for col in (GENES + CELLS):
    transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    vec_len = len(transformed_all[col].values)
    raw_vec = transformed_all[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    transformed_all[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]

Genes_Mean = transformed_all[GENES + CELLS].mean()

fig, ax = plt.subplots(figsize=(15, 5))
ax.set_xlabel ("Transformed Genes + Cells", fontsize=18)
ax.set_ylabel ("Mean of Genes + Cells", fontsize=18);

Genes_Mean.plot(lw=3);

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')

GENES = [col for col in train_features.columns if col.startswith('g-')]
transformed_Genes = train_features

for col in (GENES):
    transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    vec_len = len(transformed_Genes[col].values)
    raw_vec = transformed_Genes[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    transformed_Genes[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]

Genes_Mean = transformed_Genes[GENES + CELLS].mean()

fig, ax = plt.subplots(figsize=(15, 5))
ax.set_xlabel ("Transformed Genes Only", fontsize=18)
ax.set_ylabel ("Mean of Genes + Cells", fontsize=18);

Genes_Mean.plot(lw=3);

**Conclusion**: Compare the normal graph, we think the scale of the genes and cells are different, we decide to only apply the transfers to genes features.

2. How to handle the variance of the features?

One common used method is to apply variance threhold across the features.



In [ ]:
import pandas as pd
from sklearn.decomposition import PCA


def loading_files():

  train_features = pd.read_csv('../input/lish-moa/train_features.csv')
  train_features["cp_dose_time"] = train_features["cp_dose"] + "_" + train_features["cp_time"].astype("str")
  train_features["cp_dose_time"] = train_features["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})

  train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
  train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

  test_features = pd.read_csv('../input/lish-moa/test_features.csv')
  test_features["cp_dose_time"] = test_features["cp_dose"] + "_" + test_features["cp_time"].astype("str")
  test_features["cp_dose_time"] = test_features["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})

  sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

  return train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission

def processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission):
  
  GENES = [col for col in train_features.columns if col.startswith('g-')]
  CELLS = [col for col in test_features.columns if col.startswith('c-')]

  for col in (GENES):

    transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
  
  from sklearn.cluster import KMeans

  def fe_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 123):
      
      features_g = list(train[GENES])
      features_c = list(train[CELLS])
      
      def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
          train_ = train[features].copy()
          test_ = test[features].copy()
          data = pd.concat([train_, test_], axis = 0)
          kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
          train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
          test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
          train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
          test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
          return train, test
      
      train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
  
      return train, test

  train_features ,test_features=fe_cluster(train_features,test_features)

  def fe_stats(train, test):
    
    features_c = list(train.columns[776:876])
    
    for df in train, test:

        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        
    return train, test

  train_features,test_features=fe_stats(train_features,test_features)

  return train_features,test_features

In [ ]:
train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission = loading_files()
train_features,test_features = processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission)

# GENES

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

n_comp = 200

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

#CELLS
n_comp = 60

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
train_features_variance = pd.DataFrame(train_features[GENES].var())
train_features_variance.columns = ['Vaiance']
train_features_variance = train_features_variance.sort_values('Vaiance',ascending= False)

fig, ax = plt.subplots(figsize=(15, 5))
ax.set_xlabel ("Genes", fontsize=18)
ax.set_ylabel ("Variance of Genes", fontsize=18);

train_features_variance['Vaiance'].plot(lw=3);

In [ ]:
data = train_features[GENES]
a = pd.DataFrame(data.std()/data.mean())
a.columns = ['Ratio']
a = a.sort_values('Ratio',ascending= False)

fig, ax = plt.subplots(figsize=(15, 5))
ax.set_xlabel ("Genes", fontsize=18)
ax.set_ylabel ("Std/Mean Ratio", fontsize=18);

a['Ratio'].plot(lw=3);

**Conclusion**: After comparing the two ways for variance filtering, we think other than giving a threhold for the numbers after PCA, we also apply the models to the two tails in the Ratio graph. In the competition, we only get a chance to apply the part for the positive side. 

# Generate Pre-Trained Model

In the dataset, it contains train_targets_nonscored.csv file. This file contains additional binary MoA responses for the training data. Although the data is not predicted nor scored, the file can be used to generate pre-trained model. The file has additional features comparing with the files. K-means is used to generate the number of features needed for the input layer, as we want to explore whether to use the input layer weight and hidden layer weight, or just use the hidden layer weight. By referencing to the public score, as a conclusion, we believe that only replace the hidden layer weight can optimize the public score.

In [ ]:
#Common Structures. As the only difference between Model 1 and Model 2 is the feature engineering, so the common part is stored as functions.
#The function can also be used in the Model 1 and Model 2, so these functions will not be loaded again in the later section.

#Load 

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import QuantileTransformer

import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master/iterstrat/')
from ml_stratifiers import MultilabelStratifiedKFold

from sklearn.feature_selection import VarianceThreshold

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

def loading_files():

  train_features = pd.read_csv('../input/lish-moa/train_features.csv')
  train_features["cp_dose_time"] = train_features["cp_dose"] + "_" + train_features["cp_time"].astype("str")
  train_features["cp_dose_time"] = train_features["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})

  train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
  train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

  test_features = pd.read_csv('../input/lish-moa/test_features.csv')
  test_features["cp_dose_time"] = test_features["cp_dose"] + "_" + test_features["cp_time"].astype("str")
  test_features["cp_dose_time"] = test_features["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})

  sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

  return train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission

def processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission):
  
  GENES = [col for col in train_features.columns if col.startswith('g-')]
  CELLS = [col for col in train_features.columns if col.startswith('c-')]

  for col in (GENES):

    transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
  
  from sklearn.cluster import KMeans

  def fe_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 123):
      
      features_g = list(train[GENES])

      def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
          train_ = train[features].copy()
          test_ = test[features].copy()
          data = pd.concat([train_, test_], axis = 0)
          kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
          train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
          test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
          train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
          test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
          return train, test
      
      train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
 
      return train, test

  train_features ,test_features=fe_cluster(train_features,test_features)

  def fe_stats(train, test):
    
    features_c = list(train[CELLS])
    
    for df in train, test:

        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        
    return train, test

  train_features,test_features=fe_stats(train_features,test_features)

  return train_features,test_features

def VarianceFilter(num,train_features,test_features):
  var_thresh = VarianceThreshold(threshold=num)
  data = train_features.append(test_features)
  data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

  train_features_transformed = data_transformed[ : train_features.shape[0]]
  test_features_transformed = data_transformed[-test_features.shape[0] : ]


  train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                                columns=['sig_id','cp_type','cp_time','cp_dose'])

  train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


  test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

  test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

  return train_features,test_features

def data_cleaning(train_features,test_features):

  train = train_features.merge(train_targets_scored, on='sig_id')
  train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
  test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

  target = train[train_targets_scored.columns]
  train = train.drop('cp_type', axis=1)
  test = test.drop('cp_type', axis=1)

  return train,test,target

def createfolds(train,number):

  folds = train.copy()

  mskf = MultilabelStratifiedKFold(n_splits=number)

  for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
      folds.loc[v_idx, 'kfold'] = int(f)

  folds['kfold'] = folds['kfold'].astype(int)

  return folds

class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

def process_data(data):
    
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    
    return data

def run_training(fold, seed):
    
    #This part is not modified, as we need to generate both pretrained model,so, this function will be loaded again for the second model.
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=5e-3, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_tr, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"NonScored{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(f"NonScored{fold}_.pth"))
    model.to(DEVICE)

    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

**Generate Pre-Trained Model for First Model**

In [ ]:
train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission = loading_files()

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 205, random_state = 20).fit(train_targets_nonscored.iloc[:,1:])
kind = 'x'
train_targets_nonscored[f'clusters_{kind}'] = kmeans.labels_[:train_targets_nonscored.shape[0]]
train_targets_nonscored = pd.get_dummies(train_targets_nonscored, columns = [f'clusters_{kind}'])
new_clusters = [col for col in train_targets_nonscored.columns if col.startswith('cl')]
train_targets_scored = train_targets_nonscored[['sig_id'] + new_clusters]


train_features,test_features = processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission)

In [ ]:
# GENES

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

n_comp = 200

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

#CELLS
n_comp = 60

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
train_features,test_features = VarianceFilter(0,train_features,test_features)
train,test,target = data_cleaning(train_features,test_features)
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
folds = createfolds(train,15)

class Model(nn.Module):
    
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.11)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.11)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
  
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(0.11)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x
    
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]

# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 35
BATCH_SIZE = 128
LEARNING_RATE = 0.002
WEIGHT_DECAY = 1e-5
NFOLDS = 15
EARLY_STOPPING_STEPS = 25
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    print("Seed:",seed)
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = pd.DataFrame(predictions)

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

**Generate Pre-Trained Model for Second Model**

In [ ]:
def run_training(fold, seed):
    
    #This part is loaded again for different file names, which can be optimized with if function.
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=5e-3, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_tr, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"SecondNonScored{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(f"SecondNonScored{fold}_.pth"))
    model.to(DEVICE)

    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [ ]:
train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission = loading_files()

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 205, random_state = 20).fit(train_targets_nonscored.iloc[:,1:])
kind = 'x'
train_targets_nonscored[f'clusters_{kind}'] = kmeans.labels_[:train_targets_nonscored.shape[0]]
train_targets_nonscored = pd.get_dummies(train_targets_nonscored, columns = [f'clusters_{kind}'])
new_clusters = [col for col in train_targets_nonscored.columns if col.startswith('cl')]
train_targets_scored = train_targets_nonscored[['sig_id'] + new_clusters]


train_features,test_features = processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission)

In [ ]:
# GENES
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

n_comp = 70

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
a = pd.DataFrame(data.std()/data.mean())
a.columns = ['Vaiance']
a = a.sort_values('Vaiance',ascending= False)
lowerthrehold = a.iloc[:,0][round(len(a.iloc[:,0])*0.5)]
upperthrehold = a.iloc[:,0][round(len(a.iloc[:,0])*0.1)]
criteria = (a['Vaiance'] > lowerthrehold) & (a['Vaiance'] < upperthrehold)
b = a[criteria]
data = data[b.index]
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data))
data2 = np.append(np.power(data2[0:20],3), data2[10:],axis = 0)
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
#CELLS
n_comp = 20

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
train_features,test_features = VarianceFilter(0,train_features,test_features)
train,test,target = data_cleaning(train_features,test_features)
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
folds = createfolds(train,15)

In [ ]:
class Model(nn.Module):
    
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(0.5)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
 
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

In [ ]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128
LEARNING_RATE = 2.15e-05
WEIGHT_DECAY = 1e-5
NFOLDS = 15
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

In [ ]:
# Averaging on multiple SEEDS

SEED = [0,1,2]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    print("Seed:",seed)
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = pd.DataFrame(predictions)

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

# Submitted Soluion:

# **First Model:** 
(This is a NN model implemented by PyTorch. The Pretrained Model Non Scored folder use the pre-trained Model in the above section)

In [ ]:
#Common Structures for Model 1 and Model 2

#Load

import time
t0 = time.time()

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import QuantileTransformer

import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master/iterstrat/')
from ml_stratifiers import MultilabelStratifiedKFold

from sklearn.feature_selection import VarianceThreshold

def seed_everything(seed=34):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=34)

def loading_files():

  train_features = pd.read_csv('../input/lish-moa/train_features.csv')
  train_features["cp_dose_time"] = train_features["cp_dose"] + "_" + train_features["cp_time"].astype("str")
  train_features["cp_dose_time"] = train_features["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})

  train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
  train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

  test_features = pd.read_csv('../input/lish-moa/test_features.csv')
  test_features["cp_dose_time"] = test_features["cp_dose"] + "_" + test_features["cp_time"].astype("str")
  test_features["cp_dose_time"] = test_features["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})

  sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

  return train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission

def processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission):
  
  GENES = [col for col in train_features.columns if col.startswith('g-')]
  CELLS = [col for col in test_features.columns if col.startswith('c-')]

  for col in (GENES):

    transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
  
  from sklearn.cluster import KMeans

  def fe_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 123):
      
      features_g = list(train[GENES])
      features_c = list(train[CELLS])
      
      def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
          train_ = train[features].copy()
          test_ = test[features].copy()
          data = pd.concat([train_, test_], axis = 0)
          kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
          train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
          test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
          train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
          test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
          return train, test
      
      train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
  
      return train, test

  train_features ,test_features=fe_cluster(train_features,test_features)

  def fe_stats(train, test):
    
    features_c = list(train.columns[776:876])
    
    for df in train, test:

        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        
    return train, test

  train_features,test_features=fe_stats(train_features,test_features)

  return train_features,test_features

def VarianceFilter(num,train_features,test_features):
  var_thresh = VarianceThreshold(threshold=num)
  data = train_features.append(test_features)
  data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

  train_features_transformed = data_transformed[ : train_features.shape[0]]
  test_features_transformed = data_transformed[-test_features.shape[0] : ]


  train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                                columns=['sig_id','cp_type','cp_time','cp_dose'])

  train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


  test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

  test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

  return train_features,test_features

def data_cleaning(train_features,test_features):

  train = train_features.merge(train_targets_scored, on='sig_id')
  train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
  test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

  target = train[train_targets_scored.columns]
  train = train.drop('cp_type', axis=1)
  test = test.drop('cp_type', axis=1)

  return train,test,target

def createfolds(train,number):

  folds = train.copy()

  mskf = MultilabelStratifiedKFold(n_splits=number)

  for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
      folds.loc[v_idx, 'kfold'] = int(f)

  folds['kfold'] = folds['kfold'].astype(int)

  return folds

class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

def process_data(data):
    
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    
    return data

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    if EPOCHS > 1:
        print('Loading the Pre-Trained Model Based on Non-Scored File to train')
        model1 = Model(num_features=num_features,num_targets=num_targets,hidden_size=hidden_size,)
        model1.load_state_dict(torch.load(f"../input/pretrainedmodelnonscored/MoANNModel/NonScored{fold}_.pth"))
        model = Model(num_features=num_features,num_targets=num_targets,hidden_size=hidden_size,)
        model.dense2.weight = model1.state_dict()['dense2.weight_v']
    else:
        model = Model(num_features=num_features,num_targets=num_targets,hidden_size=hidden_size,)

    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=5e-3, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    if EPOCHS > 1:
      for epoch in range(EPOCHS):
          
          train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
          print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
          valid_loss, valid_preds = valid_fn(model, loss_tr, validloader, DEVICE)
          print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
          
          if valid_loss < best_loss:
              
              best_loss = valid_loss
              oof[val_idx] = valid_preds
              torch.save(model.state_dict(), f"FirstMFOLD{fold}_.pth")
          
          elif(EARLY_STOP == True):
              
              early_step += 1
              if (early_step >= early_stopping_steps):
                  break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    if EPOCHS > 1:
        model.load_state_dict(torch.load(f"FirstMFOLD{fold}_.pth"))
    else:
        print("Load Pre-Trained Model to predict:","Seed:",seed,"Fold:",fold)
        model.load_state_dict(torch.load(f"../input/modelforsubmissionfinal/ModelForSubmission/FirstMFOLD{fold,seed}_.pth"))
  
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [ ]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [ ]:
#This part is used for feature selection, due to the time limit, it is not used, but it can be used to reduce the number of features used.

def decision_tree_selection():
  train_features = pd.read_csv('../input/lish-moa/train_features.csv')
  train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
  
  train = train_features.merge(train_targets_scored, on='sig_id')
  train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
  target = train[train_targets_scored.columns].iloc[:,1:]

  from sklearn.model_selection import train_test_split

  target = train[train_targets_scored.columns].iloc[:,1:]
  traindata = train.iloc[:,4:876]

  X_train, X_test, y_train, y_test = train_test_split(traindata, target, test_size=0.3, random_state=42)

  from sklearn.model_selection import cross_val_score
  from sklearn.tree import DecisionTreeClassifier

  clf = DecisionTreeClassifier(random_state=0)
  empty = []

  for i in range(X_test.shape[1]-2):
    a = X_test.iloc[:,i:i+2]
    score = sum(cross_val_score(clf, a, y_test, cv=2))/2
    empty.append([list(a.columns),score])
    print([list(a.columns),score])

  empty1 = pd.DataFrame(empty)

  empty1.columns = ['ParameterNames', 'CV_Values']
  empty1 = empty1.sort_values('CV_Values',ascending= False)
  threhold = empty1.iloc[:,1][round(len(empty1.iloc[:,1])*0.2)]
  filtered = empty1[empty1['CV_Values'] > threhold]

  predictor_list = []
  for element in filtered['ParameterNames']:
    predictor_list += element

  start_predicator = list(set(predictor_list))

  return start_predicator

In [ ]:
train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission = loading_files()
train_features,test_features = processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission)

In [ ]:
# GENES

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

n_comp = 200

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
#CELLS
n_comp = 60

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
train_features,test_features = VarianceFilter(0.5,train_features,test_features)
train,test,target = data_cleaning(train_features,test_features)
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
folds = createfolds(train,15)

In [ ]:
class Model(nn.Module):
    
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.11)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.11)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
  
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(0.11)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim)) 

In [ ]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 35
BATCH_SIZE = 128
LEARNING_RATE = 0.002
WEIGHT_DECAY = 1e-5
NFOLDS = 15
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

In [ ]:
# Averaging on multiple SEEDS
import gc

SEED = [0]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    gc.collect()
    torch.cuda.empty_cache()
    print("Seed:",seed)
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = pd.DataFrame(predictions)
test3 = test

t1 = time.time()
total = t1-t0

print(total)

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

t1 = time.time()
total = t1-t0

print(total)

In [ ]:
# The result is dumpped as a variable, as the memory is cleaned constantly.
from joblib import dump, load

dump(test3, 'model1')

# **Second Model** 
(which is same structure as the first model, only difference is the way of dealing with preprocessed data.The Pretrained Model Non Scored folder use the pre-trained Model in the above section.).

In [ ]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    if EPOCHS > 1:
        print('Loading the Pre-Trained Model Based on Non-Scored File to train')
        model1 = Model(num_features=num_features,num_targets=num_targets,hidden_size=hidden_size,)
        model1.load_state_dict(torch.load(f"../input/pretrainedmodelnonscored/MoANNModel/SecondNonScored{fold}_.pth"))
        model = Model(num_features=num_features,num_targets=num_targets,hidden_size=hidden_size,)
        model.dense2.weight = model1.state_dict()['dense2.weight_v']
    else:
        model = Model(num_features=num_features,num_targets=num_targets,hidden_size=hidden_size,)

    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=5e-3, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    if EPOCHS > 1:
      for epoch in range(EPOCHS):
          
          train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
          print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
          valid_loss, valid_preds = valid_fn(model, loss_tr, validloader, DEVICE)
          print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
          
          if valid_loss < best_loss:
              
              best_loss = valid_loss
              oof[val_idx] = valid_preds
              torch.save(model.state_dict(), f"SecondMFOLD{fold}_.pth")
          
          elif(EARLY_STOP == True):
              
              early_step += 1
              if (early_step >= early_stopping_steps):
                  break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    if EPOCHS > 1:
        model.load_state_dict(torch.load(f"SecondMFOLD{fold}_.pth"))
    else:
        print("Load Pre-Trained Model to predict:","Seed:",seed,"Fold:",fold)
        model.load_state_dict(torch.load(f"../input/modelforsubmissionfinal/ModelForSubmission/SecondMFOLD{fold,seed}_.pth"))
  
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [ ]:
train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission = loading_files()
train_features,test_features = processing_files(train_features,train_targets_scored,train_targets_nonscored,test_features,sample_submission)

In [ ]:
# GENES
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

n_comp = 70

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
a = pd.DataFrame(data.std()/data.mean())
a.columns = ['Vaiance']
a = a.sort_values('Vaiance',ascending= False)
lowerthrehold = a.iloc[:,0][round(len(a.iloc[:,0])*0.5)]
upperthrehold = a.iloc[:,0][round(len(a.iloc[:,0])*0.1)]
criteria = (a['Vaiance'] > lowerthrehold) & (a['Vaiance'] < upperthrehold)
b = a[criteria]
data = data[b.index]
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data))
data2 = np.append(np.power(data2[0:20],3), data2[10:],axis = 0)
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

#CELLS
n_comp = 20

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data))
data2 = np.append(np.power(data2[0:20],3), data2[10:],axis = 0)
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
train_features,test_features = VarianceFilter(0.5,train_features,test_features)
train,test1,target = data_cleaning(train_features,test_features)
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
folds = createfolds(train,15)

In [ ]:
class Model(nn.Module):
    
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(0.5)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
 
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

In [ ]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128
LEARNING_RATE = 2.15e-05
WEIGHT_DECAY = 1e-5
NFOLDS = 10
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

In [ ]:
# Averaging on multiple SEEDS

SEED = [0]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    gc.collect()
    torch.cuda.empty_cache()
    print("Seed:",seed)
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test1[target_cols] = pd.DataFrame(predictions)

In [ ]:
dump(test1, 'model2')

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred1 = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred1[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

t1 = time.time()
total = t1-t0

print(total)

# Third Tabnet Model: 
This model is non-NN model

Notes: when the notebook is created, environmental errors is faced, so, internet is enabled to install the pytorch-tabnet package. But in the competition, the internet is disabled.

In [ ]:
!pip install pytorch-tabnet

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master/iterstrat')
from ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
import pandas as pd 
from sklearn.metrics import roc_auc_score

import os
import random
import sys
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
from sklearn.metrics import log_loss

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(42)

In [ ]:
#Data Loading

data_path = "../input/lish-moa/"
train = pd.read_csv(data_path+'train_features.csv')

#start_predicator = decision_tree_selection()
#final_predicator = ['sig_id','cp_type','cp_time','cp_dose'] + start_predicator

#train = train[final_predicator]
train.drop(columns=["sig_id"], inplace=True)

train_targets_scored = pd.read_csv(data_path+'train_targets_scored.csv')
train_targets_scored.drop(columns=["sig_id"], inplace=True)

test = pd.read_csv(data_path+'test_features.csv')
#test = test[final_predicator]
test.drop(columns=["sig_id"], inplace=True)

GENES = [col for col in train.columns if col.startswith('g-')]
CELLS = [col for col in train.columns if col.startswith('c-')]

for col in (GENES):

  transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
  vec_len = len(train[col].values)
  vec_len_test = len(test[col].values)
  raw_vec = train[col].values.reshape(vec_len, 1)
  transformer.fit(raw_vec)

  train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
  test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

from sklearn.cluster import KMeans

def fe_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 123):
      
    features_g = list(train[GENES])
    features_c = list(train[CELLS])
      
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
      
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
      #train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
      
    return train, test

train, test=fe_cluster(train, test)

def fe_stats(train, test):
    
  features_c = list(train[CELLS])
    
  for df in train, test:

      df['c_sum'] = df[features_c].sum(axis = 1)
      df['c_mean'] = df[features_c].mean(axis = 1)
      df['c_std'] = df[features_c].std(axis = 1)
      df['c_kurt'] = df[features_c].kurtosis(axis = 1)
      df['c_skew'] = df[features_c].skew(axis = 1)
        
  return train, test

train, test=fe_stats(train, test)


submission = pd.read_csv(data_path+'sample_submission.csv')

remove_vehicle = False

if remove_vehicle:
    kept_index = train['cp_type']=='trt_cp'
    train = train.loc[kept_index].reset_index(drop=True)
    train_targets_scored = train_targets_scored.loc[kept_index].reset_index(drop=True)

train["cp_type"] = (train["cp_type"]=="trt_cp") + 0
train["cp_dose"] = (train["cp_dose"]=="D1") + 0

test["cp_type"] = (test["cp_type"]=="trt_cp") + 0
test["cp_dose"] = (test["cp_dose"]=="D1") + 0

X_test = test.values

In [ ]:
MAX_EPOCH = 200
tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.3,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )

In [ ]:
from sklearn.metrics import log_loss
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score, log_loss

class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        
        return np.mean(-aux)

In [ ]:
scores_auc_all= []
test_cv_preds = []

NB_SPLITS = 7
mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
oof_preds = []
oof_targets = []
scores = []
scores_auc = []

for seed in [0]:
  print('Seed',seed)

  for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, train_targets_scored)):
      print("FOLDS : ", fold_nb)

      ## model
      X_train, y_train = train.values[train_idx, :], train_targets_scored.values[train_idx, :]
      X_val, y_val = train.values[val_idx, :], train_targets_scored.values[val_idx, :]
      model = TabNetRegressor(**tabnet_params)

      model.fit(X_train=X_train,
                y_train=y_train,
                eval_set=[(X_val, y_val)],
                eval_name = ["val"],
                eval_metric = ["logits_ll"],
                max_epochs=MAX_EPOCH,
                patience=20, batch_size=1024, virtual_batch_size=128,
                num_workers=1, drop_last=False,
                # use binary cross entropy as this is not a regression problem
                loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)

      preds_val = model.predict(X_val)
      # Apply sigmoid to the predictions
      preds =  1 / (1 + np.exp(-preds_val))
      score = np.min(model.history["val_logits_ll"])
  #     name = cfg.save_name + f"_fold{fold_nb}"
  #     model.save_model(name)
      ## save oof to compute the CV later
      oof_preds.append(preds_val)
      oof_targets.append(y_val)
      scores.append(score)

      # preds on test
      preds_test = model.predict(X_test)
      test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)

In [ ]:
aucs = [] #roc_auc_score
for task_id in range(oof_preds_all.shape[1]):
    y_true=oof_targets_all[:, task_id]
    y_score=oof_preds_all[:, task_id]
    aucs.append(roc_auc_score(y_true,y_score))
print(f"Overall AUC : {np.mean(aucs)}")
print(f"Average CV : {np.mean(scores)}")

In [ ]:
all_feat = [col for col in submission.columns if col not in ["sig_id"]]
submission[all_feat] = test_preds_all.mean(axis=0)
# set control to 0
submission.loc[test['cp_type']==0, submission.columns[1:]] = 0
TabNetResult = submission

t1 = time.time()
print(t1-t0)

In [ ]:
dump(TabNetResult, 'model3')

# Fourth Model: 
This is a NN Model, built by Keras

In [ ]:
import os
import random

import warnings
warnings.resetwarnings()
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


import tensorflow as tf
from keras.initializers import Constant
from tensorflow.keras import callbacks, losses, backend
import tensorflow_addons as tfa

from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer, StandardScaler
from statistics import mean, median

In [ ]:
os.environ['PYTHONHASHSEED'] = '1068'
np.random.seed(1068)
tf.random.set_seed(1068)
random.seed(1068)

In [ ]:
def _path(file):
    return "../input/lish-moa/" + file

In [ ]:
train = pd.read_csv(_path("train_features.csv"))
test = pd.read_csv(_path("test_features.csv"))

target = pd.read_csv(_path("train_targets_scored.csv"))
sample_sub = pd.read_csv(_path("sample_submission.csv"))

GENES = [col for col in train.columns if col.startswith('g-')]

for col in (GENES):
  transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
  vec_len = len(train[col].values)
  vec_len_test = len(test[col].values)
  raw_vec = train[col].values.reshape(vec_len, 1)
  transformer.fit(raw_vec)

  train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
  test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

from sklearn.cluster import KMeans

def fe_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 123):
      
    features_g = list(train[GENES])
    features_c = list(train[CELLS])
      
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
      
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
      #train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
      
    return train, test

train, test=fe_cluster(train, test)

def fe_stats(train, test):
    
  features_c = list(train.columns[776:876])
    
  for df in train, test:

      df['c_sum'] = df[features_c].sum(axis = 1)
      df['c_mean'] = df[features_c].mean(axis = 1)
      df['c_std'] = df[features_c].std(axis = 1)
      df['c_kurt'] = df[features_c].kurtosis(axis = 1)
      df['c_skew'] = df[features_c].skew(axis = 1)
        
  return train, test

train, test=fe_stats(train, test)


In [ ]:
target_col = target.drop('sig_id', axis=1).columns
ctl_train = train['cp_type'] == 'ctl_vehicle'
ctl_test = test['cp_type'] == 'ctl_vehicle'
no_ctl_train = np.logical_not(ctl_train)
no_ctl_test = np.logical_not(ctl_test)
sample_sub.loc[:, target_col] = 0

In [ ]:
def preprocess(train, test, target, params):
    
    
    train = train.drop('sig_id', axis=1)
    test = test.drop('sig_id', axis=1)
    target = target.drop('sig_id', axis=1)
    
    train_len = train.shape[0]
    test_len = test.shape[0]
    
    print('train shape:', train.shape)
    print('test shape:', test.shape)

    df = pd.concat([train, test], ignore_index=True)
    
    GENES = [col for col in df.columns if col.startswith('g-')]
    CELLS = [col for col in df.columns if col.startswith('c-')]
    num_cols = GENES + CELLS
    
    
    print('label encoding...')
    df['cp_type'] = df['cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
    print('finish label encoding...')

    
    print('PCA:', params['pca'])
    if params['pca']:
        g_n_comp = params['pca']['g_n_comp']
        c_n_comp = params['pca']['c_n_comp']
        g_datas = df[GENES]
        g_datas_transformed = (PCA(n_components=g_n_comp, random_state=1068).fit_transform(g_datas[GENES]))
        g_datas_transformed = pd.DataFrame(g_datas_transformed, columns=[f'pca_G-{i}' for i in range(g_n_comp)])
        df = pd.concat((df, g_datas_transformed), axis=1)
        c_datas = df[CELLS]
        c_datas_transformed = (PCA(n_components=c_n_comp, random_state=1068).fit_transform(g_datas[GENES]))
        c_datas_transformed = pd.DataFrame(c_datas_transformed, columns=[f'pca_G-{i}' for i in range(c_n_comp)])
        df = pd.concat((df, c_datas_transformed), axis=1)
        print('finish PCA.....')
        print('train shape:', df[:train_len].shape)
        print('test shape:', df[train_len:].shape)
    
    
    print('numerical_encoding:', params['numerical_encoding'])
    if params['numerical_encoding'] == 'standart_scale':
        scaler = StandardScaler()
        df[num_cols] = scaler.fit_transform(df[num_cols])
        print('finish encoding...')
    elif params['numerical_encoding'] == 'quantiletransform':
        qt = QuantileTransformer(output_distribution='normal', random_state=1068)
        df[num_cols] = qt.fit_transform(df[num_cols])
        print('finish encoding...')
    
    

    print('VarianceThreshold:', params['variance_encoding'])
    if not params['variance_encoding']:
        train = df[:train_len].values
        test = df[train_len:].values
        target = target.values
    else:
        print('do variance encoding....')
        threshold = params['variance_encoding']
        var_thresh = VarianceThreshold(threshold=threshold)
        df = var_thresh.fit_transform(df)
        print('finish variance encoding')
        print('train shape:', df[:train_len].shape)
        print('test shape:', df[train_len:].shape)
        train = df[:train_len]
        test = df[train_len:]
        target = target.values
    
    
    print('###################')
    print('finish all proccess')
    print('train shape:', train.shape)
    print('test shape:', test.shape)
    print('target shape:', target.shape)
    print('###################')
    
    return train, test, target

In [ ]:
params = {'pca': False,
         'numerical_encoding':'quantiletransform',
         'categorycal_encoding':'label_encoding',
         'variance_encoding':False
}

train_x, test_x, train_y = preprocess(train, test, target, params)

In [ ]:
p_min = 0.001
p_max = 0.999

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -backend.mean(y_true * backend.log(y_pred) + (1 - y_true) * backend.log(1 - y_pred))

In [ ]:
class MLP:
    def __init__(self, input_shape, activation, output_bias):
        
        output_bias = tf.keras.initializers.Constant(output_bias)
        
        self.model = tf.keras.Sequential([
        
        tf.keras.layers.Input(input_shape),
        
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(512, activation=activation)),
        
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.60),
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(256, activation=activation)),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.45),
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation='sigmoid', bias_initializer=output_bias))
        
        ])
        
        
        optimizer = tfa.optimizers.Lookahead(
            tfa.optimizers.AdamW(weight_decay=1e-5),
            sync_period=5
        )
        
        
        self.model.compile(optimizer=optimizer, 
                           loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0008),
                           metrics=logloss)
        
        
        
    def fit(self, train_x,train_y, val_x, val_y, epochs, batch_size, verbose):
        reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.3, patience=5, mode='min', min_lr=5e-6)
        early_stopping = callbacks.EarlyStopping(monitor='val_logloss', min_delta=5e-6, patience=5, mode='min',restore_best_weights=True)
        history = self.model.fit(train_x,
                                 train_y,
                                 epochs=epochs,
                                 validation_data=(val_x, val_y),
                                 batch_size = batch_size,
                                 verbose = verbose,
                                 callbacks = [reduce_lr, early_stopping]
                                )
        return history
    
    
    def predict(self,test_x):   
        pred = self.model.predict(test_x)
        return pred


In [ ]:
def show_loss(losses):
    for loss in losses:
        print(' -> {:.6f}'.format(loss), end='')
    print()

In [ ]:
def plot_loss(history, i, seed):
    title = 'loss fold {} in seed {}'.format(i, seed)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
def run(train_x, train_y, test_x, n_seed, n_fold, epoch, debug, verbose, output_bias, activation, plot):
    start_run = time.time()
    preds = sample_sub[:]
    val_losses = []
    if debug:
        print('run debug mode...')
        train_x = train_x[:10]
        train_y = train_y[:10]
        # test_x = test_x[:10]   
        epoch = 2
        n_fold = 2
        n_seed = 2
        plot = True
        verbose = 1
    for seed in range(1068, n_seed + 1068):
        print('###############')
        print('run_seed:', seed)
        print('###############')
        start = time.time()
        mskf = MultilabelStratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)         
        for (i, (train_idx, val_idx)) in enumerate(mskf.split(train_x, train_y), 1):
            model = MLP(train_x.shape[1], activation, output_bias)
            history = model.fit(train_x[train_idx],
                                train_y[train_idx], 
                                train_x[val_idx], 
                                train_y[val_idx],
                                epochs=epoch, 
                                batch_size=128,
                                verbose=verbose
                                )
            
            if plot:
                plot_loss(history, i, seed)
            
            print('finish run fold', i, 'with', len(history.history['loss']), 'epoch')
            print('train_loss of last 5 epoch:', end='')
            show_loss(history.history['loss'][-5:])
            print('val_loss of last 5 epoch  :', end='')
            show_loss(history.history['val_loss'][-5:])
            val_losses.append(history.history['val_loss'][-1])
            
            
            pred = model.predict(test_x)
            preds.loc[:, target_col] += pred
        
        elapsed_time = time.time() - start
        print ("time:{0}".format(elapsed_time) + "[sec]")
    
    preds.loc[:,target_col] /= n_fold * n_seed
    
    print('-------------------------')
    print('finish train and predict!')
    print('-------------------------')
    elapsed_time = time.time() - start_run
    print ('time: {0} '.format(elapsed_time) + '[sec]')
    print('loss:', end=' ')
    med_loss = (max(val_losses) + min(val_losses)) / 2
    pm_loss = max(val_losses) - med_loss
    print('{:.6f} ± {:.6f} (mean: {:.6f})'.format(med_loss, pm_loss, mean(val_losses)))
    return preds

In [ ]:
n_seed = 3
n_fold = 5
output_bias = -np.log(train_y.mean(axis=0))
verbose = 0
debug = False
epoch = 100
activation = 'swish'
plot = False
preds = run(train_x, train_y, test_x, n_seed, n_fold, epoch, debug, verbose, output_bias, activation, plot)

In [ ]:
preds.iloc[:,1:] = np.clip(preds.iloc[:,1:].values, p_min, p_max)
preds.loc[ctl_test, target_col] = 0.0

KerasSubmission = preds

dump(KerasSubmission, 'model4')

# Fifth Model:
Comparing with the first model and second model, the difference is the new files for Drug ID file (the new file is released at the later stage of the competition).

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
 
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_dir = '../input/lish-moa/'
os.listdir(data_dir)

train_features = pd.read_csv(data_dir + 'train_features.csv')
train_targets_scored = pd.read_csv(data_dir + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir + 'train_targets_nonscored.csv')
train_drug = pd.read_csv(data_dir + 'train_drug.csv')
test_features = pd.read_csv(data_dir + 'test_features.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

print('train_features: {}'.format(train_features.shape))
print('train_targets_scored: {}'.format(train_targets_scored.shape))
print('train_targets_nonscored: {}'.format(train_targets_nonscored.shape))
print('train_drug: {}'.format(train_drug.shape))
print('test_features: {}'.format(test_features.shape))
print('sample_submission: {}'.format(sample_submission.shape))

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

print('GENES: {}'.format(GENES[:10]))
print('CELLS: {}'.format(CELLS[:10]))

In [ ]:
for col in (GENES):
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

from sklearn.cluster import KMeans

def fe_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 123):
      
    features_g = list(train[GENES])
    features_c = list(train[CELLS])
      
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
      
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
  
    return train, test

train_features ,test_features=fe_cluster(train_features,test_features)

def fe_stats(train, test):
    
    features_c = list(train.columns[776:876])
    
    for df in train, test:

        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        
    return train, test

train_features,test_features=fe_stats(train_features,test_features)

In [ ]:
SEED_VALUE = 42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=SEED_VALUE)

In [ ]:
# GENES
n_comp = 600

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=SEED_VALUE).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
# CELLS
n_comp = 50

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=SEED_VALUE).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.8)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]

train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)

test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
train = train[train['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)

In [ ]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [ ]:
target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
all_target_cols = target_cols + aux_target_cols

num_targets = len(target_cols)
num_aux_targets = len(aux_target_cols)
num_all_targets = len(all_target_cols)

print('num_targets: {}'.format(num_targets))
print('num_aux_targets: {}'.format(num_aux_targets))
print('num_all_targets: {}'.format(num_all_targets))

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }

        return dct

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    return final_loss

def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    return preds

In [ ]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [ ]:
class FineTuneScheduler:
    def __init__(self, epochs):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = name.split('.')[0][-1]

            if layer_index == 5:
                continue

            param.requires_grad = False

            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)

        self.epochs_per_step = self.epochs // len(self.frozen_layers)

        # Replace the top layers with another ones
        model_new.batch_norm5 = nn.BatchNorm1d(model_new.hidden_size[3])
        model_new.dropout5 = nn.Dropout(model_new.dropout_value[3])
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(model_new.hidden_size[-1], num_targets_new))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = name.split('.')[0][-1]

                if layer_index == last_frozen_index:
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [ ]:
feature_cols = [c for c in process_data(train).columns if c not in all_target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id', 'drug_id']]
num_features = len(feature_cols)

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128

WEIGHT_DECAY = {'ALL_TARGETS': 1e-5, 'SCORED_ONLY': 3e-6}
MAX_LR = {'ALL_TARGETS': 1e-2, 'SCORED_ONLY': 3e-3}
DIV_FACTOR = {'ALL_TARGETS': 1e3, 'SCORED_ONLY': 1e2}
PCT_START = 0.1

In [ ]:
# Show model architecture
model = Model(num_features, num_all_targets)

In [ ]:
from sklearn.model_selection import KFold

def make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH):
    vc = train.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in range(SEEDS):
        kfold_col = 'kfold_{}'.format(seed_id)
        
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train[kfold_col] = train.drug_id.map(dct1)
        train.loc[train[kfold_col].isna(), kfold_col] = train.loc[train[kfold_col].isna(), 'sig_id'].map(dct2)
        train[kfold_col] = train[kfold_col].astype('int8')
        
    return train

SEEDS = 7
NFOLDS = 7
DRUG_THRESH = 18

train = make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH)
train.head()

In [ ]:
def run_training(fold_id, seed_id):
    seed_everything(seed_id)
    
    train_ = process_data(train)
    test_ = process_data(test)
    
    kfold_col = f'kfold_{seed_id}'
    trn_idx = train_[train_[kfold_col] != fold_id].index
    val_idx = train_[train_[kfold_col] == fold_id].index
    
    train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
    valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
    
    def train_model(model, tag_name, target_cols_now, fine_tune_scheduler=None):
        x_train, y_train  = train_df[feature_cols].values, train_df[target_cols_now].values
        x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols_now].values
        
        train_dataset = MoADataset(x_train, y_train)
        valid_dataset = MoADataset(x_valid, y_valid)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY[tag_name])
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                  steps_per_epoch=len(trainloader),
                                                  pct_start=PCT_START,
                                                  div_factor=DIV_FACTOR[tag_name], 
                                                  max_lr=MAX_LR[tag_name],
                                                  epochs=EPOCHS)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=0.001)

        oof = np.zeros((len(train), len(target_cols_now)))
        best_loss = np.inf
        
        if EPOCHS > 1:
            for epoch in range(EPOCHS):
                gc.collect()
                torch.cuda.empty_cache()
                if fine_tune_scheduler is not None:
                    fine_tune_scheduler.step(epoch, model)

                train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
                valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
                print(f"SEED: {seed_id}, FOLD: {fold_id}, {tag_name}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

                if np.isnan(valid_loss):
                    break

                if valid_loss < best_loss:
                    best_loss = valid_loss
                    oof[val_idx] = valid_preds
                    torch.save(model.state_dict(), f"{tag_name}_FOLD{fold_id}_.pth")

        return oof

    fine_tune_scheduler = FineTuneScheduler(EPOCHS)

    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.to(DEVICE)

    # Train on scored + nonscored targets
    train_model(pretrained_model, 'ALL_TARGETS', all_target_cols)

    # Load the pretrained model with the best loss
    pretrained_model = Model(num_features, num_all_targets)
    if EPOCHS > 1:
        pretrained_model.load_state_dict(torch.load(f"ALL_TARGETS_FOLD{fold_id}_.pth"))
    else:
        print('Load PreTrained Model,','fold_id:',fold_id)
        pretrained_model.load_state_dict(torch.load(f"../input/fifthpretrainedmodel/ALL_TARGETS_FOLD{fold_id}_.pth"))
    pretrained_model.to(DEVICE)

    # Copy model without the top layer
    final_model = fine_tune_scheduler.copy_without_top(pretrained_model, num_features, num_all_targets, num_targets)

    # Fine-tune the model on scored targets only
    oof = train_model(final_model, 'SCORED_ONLY', target_cols, fine_tune_scheduler)

    # Load the fine-tuned model with the best loss
    model = Model(num_features, num_targets)
    if EPOCHS > 1:
        model.load_state_dict(torch.load(f"SCORED_ONLY_FOLD{fold_id}_.pth"))
    else:
        print('Load PreTrained Model,','fold_id:',fold_id)
        model.load_state_dict(torch.load(f"../input/fifthpretrainedmodel/SCORED_ONLY_FOLD{fold_id}_.pth"))
    model.to(DEVICE)

    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = np.zeros((len(test_), num_targets))
    predictions = inference_fn(model, testloader, DEVICE)
    return oof, predictions

In [ ]:
def run_k_fold(NFOLDS, seed_id):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold_id in range(NFOLDS):
        oof_, pred_ = run_training(fold_id, seed_id)
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [ ]:
from time import time

# Averaging on multiple SEEDS
SEED = [0,1]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

time_begin = time()

for seed_id in SEED:
    oof_, predictions_ = run_k_fold(NFOLDS, seed_id)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

time_diff = time() - time_begin

train[target_cols] = oof
test[target_cols] = predictions

In [ ]:
from datetime import timedelta
str(timedelta(seconds=time_diff))

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0

for i in range(len(target_cols)):
    score += log_loss(y_true[:, i], y_pred[:, i])

print("CV log_loss: ", score / y_pred.shape[1])

fifthmodelresult = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

In [ ]:
dump(fifthmodelresult,'model5')

# Sixth Model: 
The structure of the model is the same as the fifth model. As the code is not fully optimized, so, only the difference with the fifth model is shown.

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
 
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
import warnings
warnings.filterwarnings('ignore')

data_dir = '../input/lish-moa/'
os.listdir(data_dir)

train_features = pd.read_csv(data_dir + 'train_features.csv')
train_targets_scored = pd.read_csv(data_dir + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir + 'train_targets_nonscored.csv')
train_drug = pd.read_csv(data_dir + 'train_drug.csv')
test_features = pd.read_csv(data_dir + 'test_features.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

print('train_features: {}'.format(train_features.shape))
print('train_targets_scored: {}'.format(train_targets_scored.shape))
print('train_targets_nonscored: {}'.format(train_targets_nonscored.shape))
print('train_drug: {}'.format(train_drug.shape))
print('test_features: {}'.format(test_features.shape))
print('sample_submission: {}'.format(sample_submission.shape))

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

print('GENES: {}'.format(GENES[:10]))
print('CELLS: {}'.format(CELLS[:10]))

for col in (GENES):
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

from sklearn.cluster import KMeans

def fe_cluster(train, test, n_clusters_g = 35, n_clusters_c = 5, SEED = 123):
      
    features_g = list(train[GENES])
    features_c = list(train[CELLS])
      
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
      
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
  
    return train, test

train_features ,test_features=fe_cluster(train_features,test_features)

def fe_stats(train, test):
    
    features_c = list(train.columns[776:876])
    
    for df in train, test:

        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        
    return train, test

train_features,test_features=fe_stats(train_features,test_features)

SEED_VALUE = 42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=SEED_VALUE)

In [ ]:
# GENES
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

n_comp = 70

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
a = pd.DataFrame(data.std()/data.mean())
a.columns = ['Vaiance']
a = a.sort_values('Vaiance',ascending= False)
lowerthrehold = a.iloc[:,0][round(len(a.iloc[:,0])*0.7)]
upperthrehold = a.iloc[:,0][round(len(a.iloc[:,0])*0.5)]
criteria = (a['Vaiance'] > lowerthrehold) & (a['Vaiance'] < upperthrehold)
b = a[criteria]
data = data[b.index]
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data))
data2 = np.append(np.power(data2[0:20],3), data2[10:],axis = 0)
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
# CELLS
n_comp = 50

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=SEED_VALUE).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.8)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]

train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)

test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
train = train[train['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)

In [ ]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [ ]:
target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
all_target_cols = target_cols + aux_target_cols

num_targets = len(target_cols)
num_aux_targets = len(aux_target_cols)
num_all_targets = len(all_target_cols)

print('num_targets: {}'.format(num_targets))
print('num_aux_targets: {}'.format(num_aux_targets))
print('num_all_targets: {}'.format(num_all_targets))

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }

        return dct

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    return final_loss

def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    return preds

import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [ ]:
class FineTuneScheduler:
    def __init__(self, epochs):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = name.split('.')[0][-1]

            if layer_index == 5:
                continue

            param.requires_grad = False

            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)

        self.epochs_per_step = self.epochs // len(self.frozen_layers)

        # Replace the top layers with another ones
        model_new.batch_norm5 = nn.BatchNorm1d(model_new.hidden_size[3])
        model_new.dropout5 = nn.Dropout(model_new.dropout_value[3])
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(model_new.hidden_size[-1], num_targets_new))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = name.split('.')[0][-1]

                if layer_index == last_frozen_index:
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

feature_cols = [c for c in process_data(train).columns if c not in all_target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id', 'drug_id']]
num_features = len(feature_cols)
num_features

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128

WEIGHT_DECAY = {'ALL_TARGETS': 1e-5, 'SCORED_ONLY': 3e-6}
MAX_LR = {'ALL_TARGETS': 1e-2, 'SCORED_ONLY': 3e-3}
DIV_FACTOR = {'ALL_TARGETS': 1e3, 'SCORED_ONLY': 1e2}
PCT_START = 0.1

In [ ]:
from sklearn.model_selection import KFold

def make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH):
    vc = train.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in range(SEEDS):
        kfold_col = 'kfold_{}'.format(seed_id)
        
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train[kfold_col] = train.drug_id.map(dct1)
        train.loc[train[kfold_col].isna(), kfold_col] = train.loc[train[kfold_col].isna(), 'sig_id'].map(dct2)
        train[kfold_col] = train[kfold_col].astype('int8')
        
    return train

SEEDS = 7
NFOLDS = 7
DRUG_THRESH = 18

train = make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH)
train.head()

In [ ]:
def run_training(fold_id, seed_id):
    seed_everything(seed_id)
    
    train_ = process_data(train)
    test_ = process_data(test)
    
    kfold_col = f'kfold_{seed_id}'
    trn_idx = train_[train_[kfold_col] != fold_id].index
    val_idx = train_[train_[kfold_col] == fold_id].index
    
    train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
    valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
    
    def train_model(model, tag_name, target_cols_now, fine_tune_scheduler=None):
        x_train, y_train  = train_df[feature_cols].values, train_df[target_cols_now].values
        x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols_now].values
        
        train_dataset = MoADataset(x_train, y_train)
        valid_dataset = MoADataset(x_valid, y_valid)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY[tag_name])
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                  steps_per_epoch=len(trainloader),
                                                  pct_start=PCT_START,
                                                  div_factor=DIV_FACTOR[tag_name], 
                                                  max_lr=MAX_LR[tag_name],
                                                  epochs=EPOCHS)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=0.001)

        oof = np.zeros((len(train), len(target_cols_now)))
        best_loss = np.inf
        
        if EPOCHS > 1:
            for epoch in range(EPOCHS):
                gc.collect()
                torch.cuda.empty_cache()
                if fine_tune_scheduler is not None:
                    fine_tune_scheduler.step(epoch, model)

                train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
                valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
                print(f"SEED: {seed_id}, FOLD: {fold_id}, {tag_name}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

                if np.isnan(valid_loss):
                    break

                if valid_loss < best_loss:
                    best_loss = valid_loss
                    oof[val_idx] = valid_preds
                    torch.save(model.state_dict(), f"{tag_name}_FOLD{fold_id}_.pth")

        return oof

    fine_tune_scheduler = FineTuneScheduler(EPOCHS)

    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.to(DEVICE)

    # Train on scored + nonscored targets
    train_model(pretrained_model, 'ALL_TARGETS', all_target_cols)

    # Load the pretrained model with the best loss
    pretrained_model = Model(num_features, num_all_targets)
    if EPOCHS > 1:
        pretrained_model.load_state_dict(torch.load(f"ALL_TARGETS_FOLD{fold_id}_.pth"))
    else:
        print('Load PreTrained Model,','fold_id:',fold_id)
        pretrained_model.load_state_dict(torch.load(f"../input/sixthpretrainedmodel/ALL_TARGETS_FOLD{fold_id}_.pth"))
    pretrained_model.to(DEVICE)

    # Copy model without the top layer
    final_model = fine_tune_scheduler.copy_without_top(pretrained_model, num_features, num_all_targets, num_targets)

    # Fine-tune the model on scored targets only
    oof = train_model(final_model, 'SCORED_ONLY', target_cols, fine_tune_scheduler)

    # Load the fine-tuned model with the best loss
    model = Model(num_features, num_targets)
    if EPOCHS > 1:
        model.load_state_dict(torch.load(f"SCORED_ONLY_FOLD{fold_id}_.pth"))
    else:
        print('Load PreTrained Model,','fold_id:',fold_id)
        model.load_state_dict(torch.load(f"../input/sixthpretrainedmodel/SCORED_ONLY_FOLD{fold_id}_.pth"))
    model.to(DEVICE)

    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = np.zeros((len(test_), num_targets))
    predictions = inference_fn(model, testloader, DEVICE)
    return oof, predictions

In [ ]:
def run_k_fold(NFOLDS, seed_id):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold_id in range(NFOLDS):
        oof_, pred_ = run_training(fold_id, seed_id)
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [ ]:
from time import time

# Averaging on multiple SEEDS
SEED = [0]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

time_begin = time()

for seed_id in SEED:
    oof_, predictions_ = run_k_fold(NFOLDS, seed_id)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

time_diff = time() - time_begin

train[target_cols] = oof
test[target_cols] = predictions

In [ ]:
from datetime import timedelta
str(timedelta(seconds=time_diff))

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0

for i in range(len(target_cols)):
    score += log_loss(y_true[:, i], y_pred[:, i])

print("CV log_loss: ", score / y_pred.shape[1])

In [ ]:
sixthModel = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

In [ ]:
test3 = load('model1')
test1 = load('model2')
TabNetResult = load('model3')
KerasSubmission = load('model4')
fifthmodelresult = load('model5')

In [ ]:
#Combined Them
testf = test3[target_cols]*0.9+ test1[target_cols]*0.1
combineresults = pd.concat((test1.iloc[:, 0], testf), axis=1)
sub = sample_submission.drop(columns=target_cols).merge(combineresults[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

The weight for each model is based on the public score as the main method to assign coresponding weight.

In [ ]:
results = (sub.iloc[:, 1:])*(0.5) + (TabNetResult.iloc[:, 1:])*(0.1) + (KerasSubmission.iloc[:, 1:])*(0.1) + ((fifthmodelresult.iloc[:, 1:])*(0.9) + (sixthModel.iloc[:, 1:])*(0.1))*(0.3)
combineresults = pd.concat((TabNetResult.iloc[:, 0], results), axis=1)

combineresults.to_csv('submission.csv',index=False)

my_sub = 'submission.csv'

import numpy as np 
import pandas as pd 
import os 
TARGET_COL = ['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor', 'acat_inhibitor', 'acetylcholine_receptor_agonist', 'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor', 'adenosine_receptor_agonist', 'adenosine_receptor_antagonist', 'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist', 'adrenergic_receptor_antagonist', 'akt_inhibitor', 'aldehyde_dehydrogenase_inhibitor', 'alk_inhibitor', 'ampk_activator', 'analgesic', 'androgen_receptor_agonist', 'androgen_receptor_antagonist', 'anesthetic_-_local', 'angiogenesis_inhibitor', 'angiotensin_receptor_antagonist', 'anti-inflammatory', 'antiarrhythmic', 'antibiotic', 'anticonvulsant', 'antifungal', 'antihistamine', 'antimalarial', 'antioxidant', 'antiprotozoal', 'antiviral', 'apoptosis_stimulant', 'aromatase_inhibitor', 'atm_kinase_inhibitor', 'atp-sensitive_potassium_channel_antagonist', 'atp_synthase_inhibitor', 'atpase_inhibitor', 'atr_kinase_inhibitor', 'aurora_kinase_inhibitor', 'autotaxin_inhibitor', 'bacterial_30s_ribosomal_subunit_inhibitor', 'bacterial_50s_ribosomal_subunit_inhibitor', 'bacterial_antifolate', 'bacterial_cell_wall_synthesis_inhibitor', 'bacterial_dna_gyrase_inhibitor', 'bacterial_dna_inhibitor', 'bacterial_membrane_integrity_inhibitor', 'bcl_inhibitor', 'bcr-abl_inhibitor', 'benzodiazepine_receptor_agonist', 'beta_amyloid_inhibitor', 'bromodomain_inhibitor', 'btk_inhibitor', 'calcineurin_inhibitor', 'calcium_channel_blocker', 'cannabinoid_receptor_agonist', 'cannabinoid_receptor_antagonist', 'carbonic_anhydrase_inhibitor', 'casein_kinase_inhibitor', 'caspase_activator', 'catechol_o_methyltransferase_inhibitor', 'cc_chemokine_receptor_antagonist', 'cck_receptor_antagonist', 'cdk_inhibitor', 'chelating_agent', 'chk_inhibitor', 'chloride_channel_blocker', 'cholesterol_inhibitor', 'cholinergic_receptor_antagonist', 'coagulation_factor_inhibitor', 'corticosteroid_agonist', 'cyclooxygenase_inhibitor', 'cytochrome_p450_inhibitor', 'dihydrofolate_reductase_inhibitor', 'dipeptidyl_peptidase_inhibitor', 'diuretic', 'dna_alkylating_agent', 'dna_inhibitor', 'dopamine_receptor_agonist', 'dopamine_receptor_antagonist', 'egfr_inhibitor', 'elastase_inhibitor', 'erbb2_inhibitor', 'estrogen_receptor_agonist', 'estrogen_receptor_antagonist', 'faah_inhibitor', 'farnesyltransferase_inhibitor', 'fatty_acid_receptor_agonist', 'fgfr_inhibitor', 'flt3_inhibitor', 'focal_adhesion_kinase_inhibitor', 'free_radical_scavenger', 'fungal_squalene_epoxidase_inhibitor', 'gaba_receptor_agonist', 'gaba_receptor_antagonist', 'gamma_secretase_inhibitor', 'glucocorticoid_receptor_agonist', 'glutamate_inhibitor', 'glutamate_receptor_agonist', 'glutamate_receptor_antagonist', 'gonadotropin_receptor_agonist', 'gsk_inhibitor', 'hcv_inhibitor', 'hdac_inhibitor', 'histamine_receptor_agonist', 'histamine_receptor_antagonist', 'histone_lysine_demethylase_inhibitor', 'histone_lysine_methyltransferase_inhibitor', 'hiv_inhibitor', 'hmgcr_inhibitor', 'hsp_inhibitor', 'igf-1_inhibitor', 'ikk_inhibitor', 'imidazoline_receptor_agonist', 'immunosuppressant', 'insulin_secretagogue', 'insulin_sensitizer', 'integrin_inhibitor', 'jak_inhibitor', 'kit_inhibitor', 'laxative', 'leukotriene_inhibitor', 'leukotriene_receptor_antagonist', 'lipase_inhibitor', 'lipoxygenase_inhibitor', 'lxr_agonist', 'mdm_inhibitor', 'mek_inhibitor', 'membrane_integrity_inhibitor', 'mineralocorticoid_receptor_antagonist', 'monoacylglycerol_lipase_inhibitor', 'monoamine_oxidase_inhibitor', 'monopolar_spindle_1_kinase_inhibitor', 'mtor_inhibitor', 'mucolytic_agent', 'neuropeptide_receptor_antagonist', 'nfkb_inhibitor', 'nicotinic_receptor_agonist', 'nitric_oxide_donor', 'nitric_oxide_production_inhibitor', 'nitric_oxide_synthase_inhibitor', 'norepinephrine_reuptake_inhibitor', 'nrf2_activator', 'opioid_receptor_agonist', 'opioid_receptor_antagonist', 'orexin_receptor_antagonist', 'p38_mapk_inhibitor', 'p-glycoprotein_inhibitor', 'parp_inhibitor', 'pdgfr_inhibitor', 'pdk_inhibitor', 'phosphodiesterase_inhibitor', 'phospholipase_inhibitor', 'pi3k_inhibitor', 'pkc_inhibitor', 'potassium_channel_activator', 'potassium_channel_antagonist', 'ppar_receptor_agonist', 'ppar_receptor_antagonist', 'progesterone_receptor_agonist', 'progesterone_receptor_antagonist', 'prostaglandin_inhibitor', 'prostanoid_receptor_antagonist', 'proteasome_inhibitor', 'protein_kinase_inhibitor', 'protein_phosphatase_inhibitor', 'protein_synthesis_inhibitor', 'protein_tyrosine_kinase_inhibitor', 'radiopaque_medium', 'raf_inhibitor', 'ras_gtpase_inhibitor', 'retinoid_receptor_agonist', 'retinoid_receptor_antagonist', 'rho_associated_kinase_inhibitor', 'ribonucleoside_reductase_inhibitor', 'rna_polymerase_inhibitor', 'serotonin_receptor_agonist', 'serotonin_receptor_antagonist', 'serotonin_reuptake_inhibitor', 'sigma_receptor_agonist', 'sigma_receptor_antagonist', 'smoothened_receptor_antagonist', 'sodium_channel_inhibitor', 'sphingosine_receptor_agonist', 'src_inhibitor', 'steroid', 'syk_inhibitor', 'tachykinin_antagonist', 'tgf-beta_receptor_inhibitor', 'thrombin_inhibitor', 'thymidylate_synthase_inhibitor', 'tlr_agonist', 'tlr_antagonist', 'tnf_inhibitor', 'topoisomerase_inhibitor', 'transient_receptor_potential_channel_antagonist', 'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist', 'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor', 'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b', 'vitamin_d_receptor_agonist', 'wnt_inhibitor']
NUM_TARGET = len(TARGET_COL)#206
data_dir = '../input/lish-moa/'
df = pd.read_csv(data_dir+'/sample_submission.csv')
public_id = list(df['sig_id'].values)
df_test = pd.read_csv(data_dir+'/test_features.csv')
test_id = list(df_test['sig_id'].values)
private_id = list(set(test_id)-set(public_id))
df_submit = pd.DataFrame(index = public_id+private_id, columns=TARGET_COL)
df_submit.index.name = 'sig_id'
df_submit[:] = 0.1 # assign a val to private test set
df_predict = pd.read_csv(my_sub)
df_submit.loc[df_predict.sig_id,:] = df_predict[TARGET_COL].values
df_submit.loc[df_test[df_test.cp_type=='ctl_vehicle'].sig_id]=0
df_submit.to_csv('submission.csv',index=True)

As the code is not fully optimized due to the time limitation, as we are trying to compete in other competitions now, but we will definitely clean up the code better later.

Reference:

1. https://www.kaggle.com/thehemen/pytorch-transfer-learning-with-k-folds-by-drug-ids
2. https://www.kaggle.com/namanj27/new-baseline-pytorch-moa
3. To be added...(As we definitely read other notebooks as well, will try to add all of the references later)